In [8]:
from ngsolve import *
from ngsolve.webgui import Draw
from netgen.occ import *   # Opencascade for geometry modeling
from netgen import meshing
from netgen.meshing import MeshPoint, Point3d, Element1D, Element0D
from ngsolve import Mesh as NGMesh

In [3]:
def get_triangle_face(vertices):
    e1 = Segment(vertices[0],vertices[1])
    e2 = Segment(vertices[1],vertices[2])
    e3 = Segment(vertices[2],vertices[0])
    return Face(Wire([e1,e2,e3]))

In [16]:
# -------------------------
# 1. 定义 bulk 几何
# -------------------------
vertices1 = [(0,1,0),(1,0,0),(0,0,0)]
vertices2 = [(0,1,0),(1,0,0),(1,1,0)]
Omega1 = get_triangle_face(vertices1)
Omega2 = get_triangle_face(vertices2)
Omega1.faces.name = "Omega1"
Omega2.faces.name="Omega2"

background = Rectangle(1, 1).Face() 
background.edges.name = "outer"  # 外边界命名

# 有孔的区域
Omega1 = background - Omega2
Omega1.faces.name = "Omega1"
Omega2.faces.name = "Omega2"

# 边命名
Omega1.edges.Min(X).name = "left"
Omega1.edges.Min(Y).name = "bottom"
Omega2.edges.Max(X).name = "right"
Omega2.edges.Max(Y).name = "top"
Omega2.edges[0].name = 'fracture'

Omega2.faces.col = (1, 0, 0) # 设置omega2显示的区域颜色


In [17]:
# -------------------------
# 2. 生成 bulk 网格
# -------------------------
geo = Glue([Omega1, Omega2])
# bulkmesh = Mesh(OCCGeometry(geo, dim=2).GenerateMesh(maxh=0.5))
bulkmesh = OCCGeometry(geo, dim=2).GenerateMesh(maxh=0.5)

In [18]:
# help(bulkmesh)

In [19]:
# ---------------------
# 3. 找出裂缝上的点并添加 1D 元素
# ---------------------
pts_idx = []
for v in ngmesh.Vertices():
    p = v.p
    if abs(p[0] + p[1] - 1) < 1e-6:
        pts_idx.append(v.nr)

# 按 x 排序
pts_idx.sort(key=lambda idx: ngmesh.Points()[idx-1][0])  # 注意 ngmesh.Points() 是从 0 开始

# 添加裂缝 1D region
idx_segment = ngmesh.AddRegion("fracture_segment", dim=1)
for i in range(len(pts_idx) - 1):
    ngmesh.Add(Element1D([pts_idx[i], pts_idx[i+1]], index=idx_segment))

# 裂缝两端作为 0D region（可选）
idx_start = ngmesh.AddRegion("start", dim=0)
idx_end = ngmesh.AddRegion("end", dim=0)
ngmesh.Add(Element0D(pts_idx[0], index=idx_start))
ngmesh.Add(Element0D(pts_idx[-1], index=idx_end))

# ---------------------
# 4. 转换成 NGSolve mesh
# ---------------------
mesh = NGSMesh(ngmesh)

# ---------------------
# 5. 定义空间（bulk + fracture）
# ---------------------
V_bulk = H1(mesh, order=1, definedon=mesh.Materials("Omega1|Omega2"))
V_frac = H1(mesh, order=1, definedon=mesh.Materials("fracture_segment"))

Draw(mesh)

NameError: name 'ngmesh' is not defined